In [1]:
### Don't mind about this 
import warnings
warnings.filterwarnings('ignore')
###

%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier # I will use the Sklearn to get kNN
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
import math
from statistics import mode
from sklearn.preprocessing import StandardScaler

In [3]:
digits_X = pd.read_csv('mnist_train.csv') #put the data in pd Frame
digits_X.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#Separate the Label from the Training Data
train_Y = digits_X['label']
train_X = digits_X.iloc[:,1:] 

In [5]:
print(train_X.shape)
print(train_Y.shape)

(60000, 784)
(60000,)


In [6]:
X = train_X.values
Y = train_Y.values

In [7]:
#Take A subset From the Values 

subset_X = X[:788,:]
subset_Y = Y[:788]

In [8]:
A = np.array([0, 1, 2])

B = np.array([[ 0,  1,  2,  3],
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11]])
print(A[:,np.newaxis].shape)
(A[:, np.newaxis] * B)

(3, 1)


array([[ 0,  0,  0,  0],
       [ 4,  5,  6,  7],
       [16, 18, 20, 22]])

### Here, we need to implement EM with multiple Dimensions. In this sense, we need to use "Multiple Dimensions Gaussians"

In [20]:
from scipy.stats import multivariate_normal

class EM:
    def __init__(self,X ,numClusters=2):
        self.X = X
        self.numClusters = numClusters
        self.covaSize = numClusters 
       
        
    
    def E_Step(self,X):
        '''
        Using this function for giving for each data point a probability for each Cluster, I have to use Gaussian 
        Equation for updating, as we are dealing with multiple dimensions
        
        - Update each data Point 
        - Normalize the data Point probabilities
        '''
        
        Prob = np.zeros((self.numClusters,X.shape[0]))
        
        Tuple = zip(self.cluster_centers, self.mixing_coeffs, self.cluster_covs)
        div = np.zeros((X.shape[0]))
        for i, (mean,mixingCofficients,cov) in enumerate(Tuple):
            #Update the Probs
            temp = mixingCofficients*multivariate_normal.pdf(X, mean, cov, allow_singular=True)
            div += temp
            Prob[i] = temp
        
            
        Prob/=div
        
        print(Prob)
        
        return Prob
    
    
    def M_Step(self):
        
  
        X=self.X
        
        for i, prob in enumerate(self.Prob): 
            Nk = prob.sum()
            
            if Nk <= 1:
                print("Singularity detected")
                
                rand = np.random.choice(X.shape[0], self.numClusters,replace=False)
                new_mean = X[rand,:]
                
                new_cov = np.eye(X.shape[1]) * self.covaSize
            
            else:
            
                 # Update the Mean,Covariance, and Mixing Cofficients

                    #Check this link: https://stackoverflow.com/questions/26089893/understanding-numpys-einsum

                new_mean = 1/Nk * (prob[:,np.newaxis]*X).sum(axis=0)

                new_cov  = (prob[:,np.newaxis,np.newaxis]*(np.einsum('ji,jk->jik',X-new_mean,X-new_mean))).sum(axis=0)

                new_cov = 1/Nk * new_cov

                #Update the global ones

            self.cluster_centers[i] = new_mean
            self.cluster_covs[i] = new_cov
            self.mixing_coeffs[i] = Nk/X.shape[0]

                
        
    
    def fit(self,itr = 100):
        
        dataPointsSize = self.X.shape[0]
        
        numDim = self.X.shape[1]
        
        rand = np.random.choice(dataPointsSize, self.numClusters,replace=False) #Generate Random values for clusters
        self.cluster_centers = self.X[rand,:]
        
        #As initialization Step, a Number of Identity functions with respect to the number of clusters
        
        self.cluster_covs = np.stack([np.eye(numDim)*self.covaSize]*self.numClusters,axis=0)
        
        #The mixing cofficents initially considered as 1/numofClusters 
        
        self.mixing_coeffs = np.full(self.numClusters,1/self.numClusters)
        
        
        for iterations in range(itr): 
            #Expectation Step
            self.Prob = self.E_Step(self.X)

            #Maximization Step
            self.M_Step()
        
        return self.cluster_centers,self.cluster_covs
    
    
    def predict(self,X):
        
        E_S = self.E_Step(X)
        
        pred = E_S.argmax(axis=0)
        
        te_pred = np.copy(X)
        
        for i,mean in enumerate(self.cluster_centers):
            te_pred[pred==i] = mean
        return prediction
        

In [21]:
np.random.seed(42)
data = np.random.random((100, 3))


In [22]:
a = EM(data)

Center,Cov = a.fit(itr=100)

[[0.48141167 0.55207567 0.47165944 0.55705684 0.56043232 0.51603328
  0.52907229 0.51636613 0.50291801 0.52045803 0.55267219 0.51013109
  0.53408908 0.54191649 0.47163472 0.54105077 0.53787212 0.50155703
  0.50284059 0.53598819 0.52550377 0.52656801 0.48649369 0.53202694
  0.47092771 0.51985906 0.53364498 0.54310932 0.5190249  0.50266887
  0.4830395  0.50596383 0.53193268 0.5285339  0.50508316 0.50985339
  0.53607867 0.47053991 0.49954347 0.47749768 0.51297076 0.51828068
  0.51953844 0.52963061 0.50549904 0.5073138  0.47672107 0.50969346
  0.53788525 0.5523505  0.56308745 0.49105819 0.51552275 0.539555
  0.52998316 0.55351411 0.48784    0.49706875 0.54704987 0.49003054
  0.5321496  0.55424133 0.53327064 0.52853577 0.5348568  0.49282077
  0.52097422 0.51564885 0.56261749 0.56518336 0.50859787 0.49489976
  0.49314482 0.5278268  0.51017751 0.47702411 0.52915008 0.51556535
  0.492163   0.48932853 0.50918482 0.54003872 0.48047716 0.55382158
  0.55479788 0.49249987 0.54898806 0.5694177  0.52

[[3.54883044e-05 9.99711792e-01 4.70478402e-04 9.99999536e-01
  9.99151004e-01 9.95480540e-01 9.97870756e-01 9.91785296e-01
  3.06057157e-03 1.34994519e-01 9.99439047e-01 8.02955605e-07
  9.99992766e-01 9.99964767e-01 1.58118932e-04 9.94186307e-01
  9.99952908e-01 3.10459696e-06 2.92252303e-04 9.99978421e-01
  9.99399028e-01 9.97993124e-01 1.20234393e-03 2.22776333e-03
  1.59212247e-03 5.19661005e-03 9.99994841e-01 9.79609544e-01
  9.96665823e-01 1.55479361e-04 1.70204170e-02 2.42539712e-03
  8.17082594e-01 9.99998123e-01 7.44350490e-01 9.71728826e-01
  9.99945260e-01 9.07057957e-05 6.46843288e-05 2.57437083e-04
  1.49905976e-04 9.98658570e-01 1.12275891e-03 8.89876755e-01
  9.98276380e-01 6.18970014e-01 1.10491577e-05 5.23390091e-01
  9.99997958e-01 9.99985220e-01 9.98257403e-01 5.74900439e-05
  4.39144563e-03 9.72437002e-01 3.74382317e-01 9.88274976e-01
  2.28035763e-01 2.34563972e-01 9.99909052e-01 1.80330881e-04
  9.99996504e-01 9.97889771e-01 1.71272092e-01 9.99998109e-01
  3.6787

In [23]:
Center

array([[0.46215844, 0.23450802, 0.51119189],
       [0.49060819, 0.77278418, 0.48235889]])

In [24]:
Cov

array([[[ 0.06726299, -0.00134232, -0.02615699],
        [-0.00134232,  0.02377701, -0.00505046],
        [-0.02615699, -0.00505046,  0.08045652]],

       [[ 0.09338466,  0.00292114,  0.01078644],
        [ 0.00292114,  0.02210932,  0.00969474],
        [ 0.01078644,  0.00969474,  0.08339128]]])